#### Instalando Extenções

In [ ]:
!pip install azure-identity
!pip install azure-storage-blob
!pip install azure-keyvault-secrets
!pip install pandas
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------ --------------------------------- 1.9/11.6 MB 39.4 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.6 MB 50.0 MB/s eta 0:00:01
   ------------------------- -------------- 7.5/11.6 MB 53.2 MB/s eta 0:00:01
   ----------------------------------- ---- 10.3/11.6 MB 59.8 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 54.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.9 MB 77.0 MB/s eta 0:00:01
   ---------------- ----------------------- 5.2/12.9 MB 66.7 MB/s eta 0:00:01
   ------------------------- -------------- 8.1/12.9 MB 64.4 MB/s eta 0:00:01
   --------------------------------- ------ 10.9/12.9 MB 59.5 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/167.9 kB ? eta -:--:--
     ------------------------------------- 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------ --------------------------------- 1.2/7.8 MB 25.5 MB/s eta 0:00:01
   ---------------- ----------------------- 3.2/7.8 MB 34.0 MB/s eta 0:00:01
   ------------------------------ --------- 5.9/7.8 MB 41.8 MB/s eta 0:00:01
   ---------------------------------------  7.8/7.8 MB 45.4 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 41.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/219.8 kB ? eta -:--:--
   ---------------------------------------- 219.8/219.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 47.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
   ------------------------------

#### Importando Extenções

In [4]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from azure.keyvault.secrets import SecretClient
from io import BytesIO

import pandas as pd
import matplotlib.pyplot as plt

#### Selecionando o Arquivo do Container

In [2]:
KEY_VAULT_URL = "https://kv-academy-01.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)
credential = DefaultAzureCredential()
 
# Criar o cliente do Blob Service
blob_service_client = BlobServiceClient(account_url="https://daniel.santos.blob.core.windows.net", credential=credential)

# Nome do segredo que queremos pegar
secret_blob_name = 'stg-academy-nome'

# Pegando o segredo correspodente
retrieved_blob_secret = client.get_secret(secret_blob_name)

# Função da documentação
def get_blob_service_client_token_credential():
    #montando a url usando o segredo, note o .value para pegar o valor e nao o objeto
    account_url = "https://" + retrieved_blob_secret.value + ".blob.core.windows.net"
    #validando a credencial
    credential = DefaultAzureCredential()
    #Criando o objeto BlobServiceClient
    blob_service_client = BlobServiceClient(account_url, credential=credential)
    #retornando o objeto
    return blob_service_client

# Instanciando um objeto usando a função
blob_service_client = get_blob_service_client_token_credential()

# Uma ação qualquer para mostrar que a conexão foi feita
for container in blob_service_client.list_containers():
    print(container.name)

academy-teste
azure-webjobs-hosts
azure-webjobs-secrets
hands-on
projeto-daniel-danillo-matheus
projeto-gustavo-jonnathan
projeto-luca-aislan
projeto-michelle-thiago
yelpinc


In [5]:
# Acessando os containers e lendo o Arquivo CSV , sem baixar na máquina
container_name = 'projeto-daniel-danillo-matheus'
nome_csv = 'transactions_train.csv'

# Obter o cliente do blob e baixar os dados
blob_client = blob_service_client.get_blob_client(container=container_name, blob=nome_csv)
blob_data = blob_client.download_blob().readall()

# Ler o arquivo CSV diretamente em um DataFrame
df = pd.read_csv(BytesIO(blob_data))


In [6]:
df.head(5)

,step,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


#### Transformação

In [ ]:
# Dicionário para renomear as colunas
colunas_renomeadas = {
    'step': 'Tempo',
    'type': 'Tipo',
    'amount': 'Valor',
    'nameOrig': 'ContaOrigem',
    'oldbalanceOrig': 'SaldoAntigoOrigem',
    'newbalanceOrig': 'SaldoNovoOrigem',
    'nameDest': 'ContaDestino',
    'oldbalanceDest': 'SaldoAntigoDestino',
    'newbalanceDest': 'SaldoNovoDestino',
    'isFraud': 'Fraude'
}

# Aplicar a renomeação no DataFrame
df = df.rename(columns=colunas_renomeadas)

df.head()

In [9]:
# Dicionário de tradução de valores
traducao_tipo = {
    'PAYMENT': 'Pagamento',
    'TRANSFER': 'Transferência',
    'CASH_OUT': 'Saque',
    'CASH_IN': 'Depósito',
    'DEBIT': 'Débito',
    'MONEY_TRANSFER': 'Transferência de Dinheiro'
}

# Substituir os valores na coluna 'Tipo' conforme o dicionário
df['Tipo'] = df['Tipo'].replace(traducao_tipo)

df.head()

,Tempo,Tipo,Valor,ContaOrigem,SaldoAntigoOrigem,SaldoNovoOrigem,ContaDestino,SaldoAntigoDestino,SaldoNovoDestino,Fraude
0,1,Pagamento,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,Pagamento,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,Transferência,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,Saque,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,Pagamento,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6351193 entries, 0 to 6351192
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Tempo               int64  
 1   Tipo                object 
 2   Valor               float64
 3   ContaOrigem         object 
 4   SaldoAntigoOrigem   float64
 5   SaldoNovoOrigem     float64
 6   ContaDestino        object 
 7   SaldoAntigoDestino  float64
 8   SaldoNovoDestino    float64
 9   Fraude              int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 484.6+ MB
